# LangChain
LangChain is a framework for developing applications powered by language models.

In [20]:
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate
from langchain.chains import LLMChain, ConversationChain, SimpleSequentialChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

### Set up API Key

In [2]:
# Load OpenAI API key from env
load_dotenv(find_dotenv(), override=True)

True

### LLM
Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs.<br/>
There are lots of LLM providers (OpenAI, Cohere, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

In [2]:
llm = OpenAI(temperature=0.7)
print(llm.predict("Suggest a name for new mexican restaurant"))



Taco Fiesta


In [18]:
print(llm.predict("Suggest a name for new mexican restaurant"))



Casa de Fuego Mexican Grill


### Prompts
A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

LangChain provides several classes and functions to help construct and work with prompts.

1. Prompt templates: Parametrized model inputs
2. Example selectors: Dynamically select examples to include in promptsompts

#### Prompt Templates
Prompt templates are pre-defined recipes for generating prompts for language models.

A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.

LangChain provides tooling to create and work with prompt templates.

LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models.

Typically, language models expect the prompt to either be a string or else a list of chat messages.

In [3]:
joke_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
joke_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [5]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False),
 HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]

In [6]:
template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
template.format_messages(text='i dont like eating tasty things.')

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat.", additional_kwargs={}),
 HumanMessage(content='i dont like eating tasty things.', additional_kwargs={}, example=False)]

### Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

LangChain provides the Chain interface for such "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chain

Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.s.

In [7]:
chain = LLMChain(llm=llm, prompt=joke_template, verbose=True)
print(chain.run(adjective="funny", content="dog"))



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a funny joke about dog.

> Finished chain.


Q: What did the dog say when he sat on sandpaper?
A: Ruff!


In [5]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [9]:
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



- Chicken Tikka Masala
- Saag Paneer
- Lamb Vindaloo
- Vegetable Samosas
- Aloo Gobi
- Tandoori Chicken
- Naan Bread
- Chana Masala
- Gulab Jamun
- Mango Lassi


In [10]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)
food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [11]:
chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [28]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': " \n\nMaharaja's Palace Cuisine",
 'menu_items': '\n\n- Vegetable Samosas\n- Palak Paneer\n- Dal Makhani\n- Tandoori Chicken\n- Lamb Curry\n- Chicken Tikka Masala\n- Naan Bread\n- Raita\n- Basmati Rice\n- Gulab Jamun\n- Kulfi'}

### Agents

In [14]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the difference GDP of US in 2021 & 2022?")



> Entering new AgentExecutor chain...
 I need to find the GDP of the US for both years
Action: Search
Action Input: US GDP 2021 2022
Observation: $25.46 trillion
Thought: I need to calculate the difference between the two years
Action: Calculator
Action Input: 25.46 trillion - 24.22 trillion
Observation: Answer: 12400000000
Thought: I now know the final answer
Final Answer: The difference in GDP between 2021 and 2022 was $1.24 trillion.

> Finished chain.


'The difference in GDP between 2021 and 2022 was $1.24 trillion.'

In [4]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What will be his age in August 2040?")



> Entering new AgentExecutor chain...
 I need to find out when Elon Musk was born and then calculate his age in 2040.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX;  angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$226 billion as of September 2023, according to the Bloomberg Billionaires Index, and $249 billion according to Forbes, primarily from his ownership stakes in both Tesla and SpaceX.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadia

'Elon Musk will be 69 years old in August 2040.'

### Memory

In [6]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Palacio


In [7]:
name = chain.run("Indian")
print(name)



"The Spice of India"


In [9]:
type(chain.memory)

NoneType

In [12]:
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"Cielo Azul Mexican Cuisine"


In [13]:
name = chain.run("Arabic")
print(name)



Majestic Medina Cuisine


In [14]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"Cielo Azul Mexican Cuisine"
Human: Arabic
AI: 

Majestic Medina Cuisine


In [15]:
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [16]:
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was won by the West Indies in 1975. They defeated Australia by 17 runs in the final at Lord's Cricket Ground in London."

In [17]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10.'

In [18]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the 1975 Cricket World Cup was Clive Lloyd of the West Indies.'

In [19]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia by 17 runs in the final at Lord's Cricket Ground in London.
Human: How much is 5+5?
AI:  5+5 is equal to 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team in the 1975 Cricket World Cup was Clive Lloyd of the West Indies.


In [21]:
memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975.'

In [22]:
convo.run("How much is 5+5?")

' 10.'

In [23]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."

In [24]:
print(convo.memory.buffer)

Human: Who was the captain of the winning team?
AI:  I'm sorry, I don't know.
